# TensorFlow Playground
## January 2023
### by Michelle (Chelle) Davies
This notebook is my environment to practive using TensorFlow's features. Eventually, I will build more specific projects. For now, there's no (intentional) cohesive narrative with these datasets.

In [1]:
# imports
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
import opencv

ModuleNotFoundError: No module named 'tensorflow'

## Starting with a tutorial on the basics
Source: https://www.tensorflow.org/tutorials/quickstart/beginner

In [ ]:
# load the preloaded dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
mnist

Next, I'm going to build a model. These are the options:
1. Keras Sequential Model
2. Keras Functional API

I'm building a `tf.keras.Sequential` model.

In [ ]:
# Build a tf.keras.Sequential model:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [ ]:
# get predictions
predictions = model(x_train[:1]).numpy()
predictions

In [ ]:
# The tf.nn.softmax function converts these logits to probabilities for each class:
probabilities = tf.nn.softmax(predictions).numpy()
probabilities

In [ ]:
# Define a loss function for training using losses.SparseCategoricalCrossentropy:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

Before training, configure and compile the model using Keras `Model.compile`. Set the optimizer class to adam, set the loss to the `loss_fn` function defined earlier, and specify a metric to be evaluated for the model by setting the metrics parameter to accuracy.

In [ ]:
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

### Train and evaluate the model
Use the Model.fit method to adjust the model parameters and minimize the loss:

In [ ]:
model.fit(x_train, y_train, epochs=5)

The `Model.evaluate` method checks the model's performance, usually on a validation set or test set.

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

## My Own Experiment - OCR
Now, I'm going to make a project with my own data and exploration. I am going to explore creating OCR models with Tensorflow and Keras.

*Optical character recognition or optical character reader is the electronic or mechanical conversion of images of typed, handwritten or printed text into machine-encoded text, whether from a scanned document, a photo of a document, a scene-photo or from subtitle text superimposed on an image.*

In [ ]:
import numpy as np

In [ ]:
def load_mnist_dataset():
  # load data from tensorflow framework
  ((trainData, trainLabels), (testData, testLabels)) = mnist.load_data() 
  # Stacking train data and test data to form single array named data
  data = np.vstack([trainData, testData]) 
  # Vertical stacking labels of train and test set
  labels = np.hstack([trainLabels, testLabels]) 
  # return a 2-tuple of the MNIST data and labels
  return (data, labels)

In [ ]:
def load_az_dataset(datasetPath):
    # List for storing data
    data = []
    # List for storing labels
    labels = []
    for row in open(datasetPath): #Openfile and start reading each row
        #Split the row at every comma
        row = row.split(",")
        #row[0] contains label
        label = int(row[0])
        #Other all collumns contains pixel values make a saperate array for that
        image = np.array([int(x) for x in row[1:]], dtype="uint8")
        #Reshaping image to 28 x 28 pixels
        image = image.reshape((28, 28))
        #append image to data
        data.append(image)
        #append label to labels
        labels.append(label)
    #Converting data to numpy array of type float32
    data = np.array(data, dtype='float32')
    #Converting labels to type int
    labels = np.array(labels, dtype="int")
    return (data, labels)

In [ ]:
(digitsData, digitsLabels) = load_mnist_dataset()
(azData, azLabels) = load_az_dataset('hwData/A_Z Handwritten Data.csv')

In [ ]:
# the MNIST dataset occupies the labels 0-9, so let's add 10 to every A-Z label to ensure the A-Z characters are not incorrectly labeled 
azLabels += 10

In [ ]:
# stack the A-Z data and labels with the MNIST digits data and labels
data = np.vstack([azData, digitsData])
labels = np.hstack([azLabels, digitsLabels])

In [ ]:
# Each image in the A-Z and MNIST digts datasets are 28x28 pixels;
# However, the architecture we're using is designed for 32x32 images,
# So we need to resize them to 32x32
data = [cv2.resize(image, (32, 32)) for image in data]
data = np.array(data, dtype="float32")

In [ ]:
# add a channel dimension to every image in the dataset and scale the
# pixel intensities of the images from [0, 255] down to [0, 1]
data = np.expand_dims(data, axis=-1)
data /= 255.0

In [ ]:
le = LabelBinarizer()
labels = le.fit_transform(labels)

In [ ]:
counts = labels.sum(axis=0)

In [ ]:
# account for skew in the labeled data
classTotals = labels.sum(axis=0)
classWeight = {}

In [ ]:
# loop over all classes and calculate the class weight
for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]

In [ ]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(
rotation_range=10,
zoom_range=0.05,
width_shift_range=0.1,
height_shift_range=0.1,
shear_range=0.15,
horizontal_flip=False,
fill_mode="nearest")

In [ ]:
# imports
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K

In [ ]:
# Resnet class
